<a href="https://colab.research.google.com/github/vidyameidita/crudVidya/blob/main/Movie_Recommendation_Vidya_Meidita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv',
                        usecols=['movieId', 'title'], dtype={'movieId': 'int32', 'title': 'str'})

In [3]:
ratings_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv',
                         usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [4]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
movies_df.shape

(9742, 2)

In [6]:
ratings_df.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [7]:
ratings_df.shape

(25000095, 3)

In [8]:
df = pd.merge(ratings_df, movies_df, on = 'movieId')

In [9]:
df

,userId,movieId,rating,title
0,1,296,5.0,Pulp Fiction (1994)
1,1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994)
2,1,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993)
3,1,665,5.0,Underground (1995)
4,1,899,3.5,Singin' in the Rain (1952)
...,...,...,...,...
23972480,162541,50872,4.5,Ratatouille (2007)
23972481,162541,55768,2.5,Bee Movie (2007)
23972482,162541,56176,2.0,Alvin and the Chipmunks (2007)
23972483,162541,58559,4.0,"Dark Knight, The (2008)"


In [10]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.groupby(by=['title'])['rating']
                     .count().reset_index().rename(columns = {'rating': 'totalRatingCount'})
                     [['title', 'totalRatingCount']])
movie_ratingCount

,title,totalRatingCount
0,'71 (2014),333
1,'Hellboy': The Seeds of Creation (2004),99
2,'Round Midnight (1986),188
3,'Salem's Lot (2004),161
4,'Til There Was You (1997),475
...,...,...
9444,eXistenZ (1999),5580
9445,xXx (2002),5956
9446,xXx: State of the Union (2005),1133
9447,¡Three Amigos! (1986),6414


In [11]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,296,5.0,Pulp Fiction (1994),79672
1,1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),7058
2,1,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),6616
3,1,665,5.0,Underground (1995),1269
4,1,899,3.5,Singin' in the Rain (1952),10895


In [12]:
popularity_threshold = 50
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,296,5.0,Pulp Fiction (1994),79672
1,1,306,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),7058
2,1,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),6616
3,1,665,5.0,Underground (1995),1269
4,1,899,3.5,Singin' in the Rain (1952),10895


In [13]:
rating_popular_movie.shape

(23953344, 5)

In [14]:
# Misal tingkat popularitas yang lebih tinggi untuk mengurangi ukuran pivot table
popularity_threshold = 1000
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
movie_features_df = rating_popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
print("Contoh movie_features_df:")
print(movie_features_df.head())

Contoh movie_features_df:
userId                          1       2       3       4       5       \
title                                                                    
'burbs, The (1989)                 0.0     0.0     0.0     0.0     0.0   
(500) Days of Summer (2009)        0.0     0.0     0.0     0.0     0.0   
*batteries not included (1987)     0.0     0.0     0.0     0.0     0.0   
...And Justice for All (1979)      0.0     0.0     0.0     0.0     0.0   
10 Cloverfield Lane (2016)         0.0     0.0     0.0     0.0     0.0   

userId                          6       7       8       9       10      ...  \
title                                                                   ...   
'burbs, The (1989)                 0.0     0.0     0.0     0.0     0.0  ...   
(500) Days of Summer (2009)        0.0     0.0     0.0     0.0     0.0  ...   
*batteries not included (1987)     0.0     0.0     0.0     0.0     0.0  ...   
...And Justice for All (1979)      0.0     0.0     0.0     0

In [15]:
from scipy.sparse import csr_matrix

In [16]:
movie_features_df_matrix = csr_matrix(movie_features_df.values)

In [17]:
from sklearn.neighbors import NearestNeighbors

In [18]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [19]:
 movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,162532,162533,162534,162535,162536,162537,162538,162539,162540,162541
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
*batteries not included (1987),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)

2717


In [21]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,162532,162533,162534,162535,162536,162537,162538,162539,162540,162541
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
*batteries not included (1987),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
movie_features_df.shape

(3775, 162539)

In [28]:
for i in range (0, len(distances.flatten())):
  if i == 0:
    print('Recommendition for (0) :\n'.format(movie_features_df.index[449]))
  else:
          print('{0} {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendition for (0) :

1 Revolutionary Road (2008), with distance of 0.6309784650802612:
2 Doubt (2008), with distance of 0.6563894748687744:
3 Changeling (2008), with distance of 0.6633532047271729:
4 Education, An (2009), with distance of 0.6720770597457886:
